<a href="https://colab.research.google.com/github/ralik45/Rice-Classification/blob/main/Model_for_Rice_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'rice-image-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2049052%2F3399185%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240810%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240810T092404Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D44e9fa1d3057f2bdf8138d56e05ee5e90e80beb3f5d109b3cb1d8312e135cbb1fd63a611f1aaafe237d49035636ebfcae7f66dc47b8b1043f81c18f93d7615fe8444ed359506de08555b237d9901dc49bb895fc911a06623bc1ab0b1cba6c4031935fb91613cba89dc7f9f59a32eec2f4d5b95f60f3ffd9750ad2f73e0c1f7cf1eacd77c2141233e34ffec7edc74e7de4eb7b6010b7063092d31732eed1f527253508db548a04ae6fc1e3404ea5125d2a8df985220d1eedc90a566b36f02aa66c522a56f1e88f74bf32743712baa699b2ff40a9a66bd0df98ec77446b457a6750bb083c231b4c879543cbf4d2f358e04435a9af187ce1de401f6fd7845f653a0,testing-rice:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3399927%2F5920253%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240810%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240810T092404Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0ead01328947acb8f9e35ae75d39fd4fa170eb83db7519d8ae2719c887b72f0630b5c7898d55fecd3c4abf05b62bb5d72af115b2ba8d5b0d3f4de73b15dafc72e3e2f1db5331619b1aca676a9c5551f8696256b0ddfd4938fa43b10f199063c7ff919642d1889b84767641b72387b2d9a13fbdbbe7ada1f63dc3d59819fdc16d1dc2b6ce7ae6d69d8d52f057373bd8d5a2e7055f2ebe5173abf93288f6547355a09a35f9178e442bb1da78d2716d1e6ec65d74e7991dba3abcfdcde488971bdde0cd3f6b85dc1af7dae11b74607b475a13576f93914b091b4344bca51f3e949949b706cdef011a90bb8bc9072a13066d122ff7dc354b96422d40fb7fd7494537'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Library

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import os
import pathlib
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model, callbacks
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Convolution2D,Activation,Flatten,Dense,Dropout,MaxPool2D,BatchNormalization

# Dataset

In [ ]:
dataset = '/kaggle/input/rice-image-dataset/Rice_Image_Dataset'
image_classes = os.listdir(dataset)
image_classes

In [ ]:
rice_types = ['Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag']
images = []

fig, axs = plt.subplots(1, len(rice_types), figsize=(15, 5))

for i, rice_type in enumerate(rice_types):
    img = cv2.imread(f'/kaggle/input/rice-image-dataset/Rice_Image_Dataset/{rice_type}/{rice_type} (1).jpg')
    images.append(img)
    axs[i].imshow(img)
    axs[i].set_title(rice_type)
    print(img.shape)

plt.show()

In [ ]:
dataGenerator = ImageDataGenerator(rescale= 1. / 255, validation_split=0.2)
train_data = dataGenerator.flow_from_directory(
    dataset,
    target_size=(250,250),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    shuffle=True,
)
val_data = dataGenerator.flow_from_directory(
    dataset,
    target_size=(250,250),
    batch_size=64,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

# Model

## Create Model

In [ ]:
CNN = Sequential()

CNN.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape = (250,250,3)))
CNN.add(BatchNormalization())
CNN.add(Activation('relu'))
CNN.add(Conv2D(128, kernel_size=(3,3), padding='same', input_shape = (250,250,3)))
CNN.add(BatchNormalization())
CNN.add(Activation('relu'))
CNN.add(MaxPool2D(pool_size=(3, 3)))

CNN.add(Conv2D(128, kernel_size=(3,3), padding='same', input_shape = (250,250,3)))
CNN.add(BatchNormalization())
CNN.add(Activation('relu'))
CNN.add(Conv2D(128, kernel_size=(3,3), padding='same', input_shape = (250,250,3)))
CNN.add(BatchNormalization())
CNN.add(Activation('relu'))

CNN.add(Conv2D(256, kernel_size=(3,3), padding='same', input_shape = (250,250,3)))
CNN.add(BatchNormalization())
CNN.add(Activation('relu'))
CNN.add(MaxPool2D(pool_size=(3, 3)))

CNN.add(Conv2D(512, kernel_size=(3,3), padding='same', input_shape = (250,250,3)))
CNN.add(BatchNormalization())
CNN.add(Activation('relu'))
CNN.add(MaxPool2D(pool_size=(3, 3)))

CNN.add(Conv2D(512, kernel_size=(3,3), padding='same', input_shape = (250,250,3)))
CNN.add(BatchNormalization())
CNN.add(Activation('relu'))

CNN.add(Conv2D(512, kernel_size=(3,3), padding='same', input_shape = (250,250,3)))
CNN.add(BatchNormalization())
CNN.add(Activation('relu'))
CNN.add(MaxPool2D(pool_size=(3, 3)))

CNN.add(Flatten())
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.1))
CNN.add(Dense(256, activation='relu'))
CNN.add(Dropout(0.1))
CNN.add(Dense(5, activation='softmax'))

CNN.summary()

In [ ]:
CNN.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Train Model

In [ ]:
early_stopping_cb = callbacks.EarlyStopping(monitor="val_loss", patience=3)
reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss",
                                              factor=0.2,
                                              patience=2,
                                              verbose=1,
                                              min_lr=1e-7)

checkpoint_path = "./Checkpoint/"
model_checkpoint = callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_weights_only=True,
                                                      save_best_only=True,
                                                      monitor="val_loss")

history = CNN.fit(train_data,
                    epochs=3,
                    validation_data=val_data,
                    callbacks=[early_stopping_cb, model_checkpoint, reduce_lr])

# Evaluate

## Loss & Accuracy

In [ ]:
def plot_training_curves(history_df):
    plt.figure(figsize = (13, 4), dpi = 120)
    ax = plt.subplot(1, 2, 1)
    plt.plot(range(1, len(history_df) + 1), history_df['loss'], marker = '.', label = 'Training Loss')
    plt.plot(range(1, len(history_df) + 1), history_df['val_loss'], marker = '^', label = 'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy')
    plt.grid()
    plt.legend()
    ax = plt.subplot(1, 2, 2)
    plt.plot(range(1, len(history_df) + 1), history_df['accuracy'], marker = '.', label = 'Training Accuracy')
    plt.plot(range(1, len(history_df) + 1), history_df['val_accuracy'], marker = '^', label = 'Validation Accurcay')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
plot_training_curves(pd.DataFrame(history.history))

## Classification Report

In [ ]:
class_dict = val_data.class_indices
li = list(class_dict.keys())
print(li)

In [ ]:
from sklearn.metrics import classification_report
y_pred = CNN.predict(val_data)
y_pred = np.argmax(y_pred, axis=1)
print('Classification Report')
target_names =li
print(classification_report(val_data.classes, y_pred, target_names=target_names))

## Confusion Matrix

In [ ]:
import seaborn as sns
plt.figure(figsize = (6,6), dpi = 100)
sns.heatmap(confusion_matrix(val_data.classes, y_pred), annot = True, cmap = 'Reds')
plt.xlabel('Predictions')
plt.ylabel('True Labels')
plt.title('Conusion Matrix')
plt.show()

## Test Images

In [ ]:
from PIL import Image
from keras.models import load_model
# Load the trained model
model = CNN

# Define the image size and preprocessing function
img_size = (250, 250)
def preprocess_image(img_path):
    img = Image.open(img_path).convert('RGB')
    img = img.resize(img_size)
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Set the path to the image folder
test_dir = '/kaggle/input/testing-rice/test_dir'
img_folder = test_dir
class_names = list(train_data.class_indices.keys())
# Loop through all the images in the folder and make predictions
for filename in os.listdir(img_folder):
    img_path = os.path.join(img_folder, filename)
    img = preprocess_image(img_path)
    pred = model.predict(img)
    class_idx = np.argmax(pred)
    class_name = class_names[class_idx]
    print(f'Image: {filename}, Class: {class_name}')